In [10]:
! pwd
! pip uninstall sympytorch -y

/share_zeta/Proxy-Sim/PhysicsSimulationDeepLearning


Found existing installation: sympytorch 0.1.4
Uninstalling sympytorch-0.1.4:
  Successfully uninstalled sympytorch-0.1.4


In [1]:
import os
import sys


ROOT="/home/guillermo.carrilho/PhysicsSimulationDeepLearning"
#ROOT="/share_zeta/Proxy-Sim/PhysicsSimulationDeepLearning"

sys.path.append(os.path.join(ROOT,"Physical_models"))

Equation solve by this algorithm is:
$$\textbf{u}=-\frac{\textbf{k}}{\mu}(\nabla p - \rho g)$$

### The case of this implementation is

$$\phi \frac{\partial p}{\partial t} - \frac{1}{\mu} \nabla \cdot (\textbf{k} \nabla p) = q$$

# stable fluids approach

Equation solve by this algorithm is:
$$\textbf{u}=-\frac{\textbf{k}}{\mu}(\nabla p - \rho g)$$

### The case of this implementation is

$$\phi \frac{\partial p}{\partial t} - \frac{1}{\mu} \nabla \cdot (\textbf{k} \nabla p) = q$$

In [5]:
#from Differentiable_simulation import dK_w,K_w,K_o,grad_phi_dK
from phi.torch.flow import *
from phi.torch.flow import diffuse, advect, Solve, fluid, math,Field, unstack,stack,batch,field,vec

class darcy_flow_model(object):
  def __init__(self,field,dt):
    self.field=field
    self.dt=dt

  def nK(self,domain):
    return 0.001*math.exp(-math.sum(((domain.center-0.5)**2),"vector")/(2*0.5))

  def reaction_diffusion(self, u, dt):
    su = self.nK(u.geometry) * field.laplace(u)
    return u + dt * su
  def step(self,v):
    return self.reaction_diffusion(v,self.dt)
  
args={"k":0.1}
  

geo=UniformGrid(x=60, y=60,bounds=Box(x=1.0, y=1.0))
phi_w=Field( geo,values=vec(x=0.0,y=0.0),
      boundary= {
          'x-': vec(x=0.1,y=0.0),
          'x+': ZERO_GRADIENT,
          'y-': 0,
          'y+': 0
 })

phy=physical_model(
   phi_w,0.01
)

NameError: name 'physical_model' is not defined

## 2D

In [ ]:
geo=UniformGrid(x=20, y=20,bounds=Box(x=1.0, y=1.0))
phi_w=Field( geo,values=tensor(0.0),
      boundary= {
          'x-': tensor(0.01),
          'x+': ZERO_GRADIENT,
          'y-': 0,
          'y+': 0
 })
phi_w
model=darcy_flow_model()

In [3]:
class simple_cnn_model(torch.nn.Module):
  def __init__(self):
    super(simple_cnn_model,self).__init__()

    self.conv1=torch.nn.Conv2d(2, 32, (9,9), padding=4)
    self.conv2=torch.nn.Conv2d(32, 64, (9,9), padding=4)
    self.conv3=torch.nn.Conv2d(64, 2, (9,9), padding=4)
    self.act1=torch.nn.ReLU()
    self.act2=torch.nn.ReLU()
    self.act3=torch.nn.Tanh()
  def forward(self,x):
    x=self.act1(self.conv1(x))
    x=self.act2(self.conv2(x))
    x=self.act3(self.conv3(x))*1
    #x=torch.clamp(x, min=-0.5, max=0.5)
    return x[0]

In [ ]:
model=simple_cnn_model().train()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)

In [17]:
v,p = fluid.make_incompressible(v0,solve=Solve('CG-adaptive',1e-2,1e-2))

/usr/local/lib/python3.10/dist-packages/phiml/math/_optimize.py:640: UserWarning: Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).
  warnings.warn("Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).")


In [14]:
from Train_differentiable_physics import PINNS_based_SOL_trainer
import torch
T=PINNS_based_SOL_trainer(
    field=(v0,fluid.make_incompressible(v0,solve=Solve('CG-adaptive',1e-2,1e-2))[-1]),
    model=model,
    optimizer=optimizer,
    simulation_steps=2,
    time_step=0.1,
    loss=torch.mean
    )

In [16]:
from random import randint
randint()

TypeError: Random.randint() missing 2 required positional arguments: 'a' and 'b'

In [15]:
T.train(5)

[array(0.5626921, dtype=float32),
 array(-0.45075464, dtype=float32),
 array(-0.6038111, dtype=float32),
 array(-0.64447063, dtype=float32),
 array(-0.6510437, dtype=float32)]

## 3D

/usr/local/lib/python3.10/dist-packages/phiml/math/_optimize.py:640: UserWarning: Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).
  warnings.warn("Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).")


Diverged: Solve diverged within 819 iterations using Φ-ML CG (PyTorch*)

In [4]:
boundary = {'x-': vec(x=1.0,y=0.0,z=0.0),
            'x+': ZERO_GRADIENT,
            'y-':0,
            'y+':0,
            'z-':0,
            'z+':0
            }
boundary
geo=UniformGrid(x=128, y=128,z=128,bounds=Box(x=50, y=50,z=50))
v0=Field(geo,values=vec(x=tensor(0.0),y=tensor(0.0),z=tensor(0.0)),boundary=boundary)

phys=darcy_flow_model(v0,dt=0.1,
                    #advection_solver=(lambda v: Solve('CG-adaptive',1e-2,1e-2,x0=v)),projection_solver=Solve('CG-adaptive',1e-2,1e-2)
                    )

TypeError: darcy_flow_model.__init__() got an unexpected keyword argument 'dt'

In [125]:
phys.step(v0)

/usr/local/lib/python3.10/dist-packages/phiml/math/_optimize.py:640: UserWarning: Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).
  warnings.warn("Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).")


Field[(xˢ=128, yˢ=128, zˢ=128, vectorᶜ=x,y,z)]

In [130]:
v0.geometry

(xˢ=128, yˢ=128), bounds=Box(x=(0, 50), y=(0, 50))

# Differentiable physics with PINNs

## Linear Darcy model

with
$$K(x)=\exp((p-p_0)^2)$$

### 2D

#### Solver

In [6]:
from phi.torch.flow import *
class darcy_flow_model(object):
  def __init__(self,field,dt):
    self.field=field
    self.dt=dt

  def nK(self,domain):
    return 0.001*math.exp(-math.sum(((domain.center-0.5)**2),"vector")/(2*0.5))

  def reaction_diffusion(self, u, dt):
    su = self.nK(u.geometry) * field.laplace(u)
    return u + dt * su
  def step(self,v):
    return self.reaction_diffusion(v,self.dt)
args={"k":0.1}


In [7]:
geo=UniformGrid(x=20, y=20,bounds=Box(x=1.0, y=1.0))
phi_w=Field( geo,values=tensor(0.0),
      boundary= {
          'x-': tensor(0.01),
          'x+': ZERO_GRADIENT,
          'y-': 0,
          'y+': 0
 })
phi_w
model=darcy_flow_model(phi_w,0.1)

In [8]:
PHI=[]
for i in range(100):
  phi_w=model.reaction_diffusion(phi_w,0.01)
  PHI.append(phi_w)

In [ ]:
for i in PHI[::10]:
  plot(i)

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


RuntimeError: No user interface available.

#### Solver in the loop

In [9]:
from Train_differentiable_physics_with_PINNs import PINNS_based_SOL_trainer

In [ ]:
import torch
from einops import rearrange
import matplotlib.pyplot as plt

class simple_cnn_model(torch.nn.Module):
  def __init__(self):
    super(simple_cnn_model,self).__init__()

    self.conv1=torch.nn.Conv2d(1, 32, (9,9), padding=4)
    self.conv2=torch.nn.Conv2d(32, 64, (9,9), padding=4)
    self.conv3=torch.nn.Conv2d(64, 1, (9,9), padding=4)
    self.act1=torch.nn.ReLU()
    self.act2=torch.nn.ReLU()
    self.act3=torch.nn.Tanh()
  def forward(self,x):
    x=self.act1(self.conv1(x))
    x=self.act2(self.conv2(x))
    x=self.act3(self.conv3(x))*0.01
    #x=torch.clamp(x, min=-0.5, max=0.5)
    return x
  

#class composed_loss(object):

from DL_models.PINNS.utils import x_grad

def composed_loss(p,x):
  """
  u: [b x y p]
  x: [b x y (x y t)]
  """
  p=rearrange(p,"b x y-> b (x y)")
  x=rearrange(x,"b x y p-> b (x y) p")
  dpdt=x_grad(p,x,0,1)[...,2]
  print(x_grad(p,x,0,2)[...,:2].shape)
  lap_u=torch.sum(x_grad(p,x,0,2)[...,:2],axis=-1)
  k=0.001*torch.exp(
    -torch.sum(((x[...,:2]-torch.mean(x[...,:2]))**2),"vector")/(2*0.5))

  return torch.mean(dpdt-k*lap_u)

SyntaxError: incomplete input (2844890087.py, line 23)

In [12]:
st_model=simple_cnn_model()
optimizer=torch.optim.Adam(st_model.parameters(),lr=1e-4)

In [ ]:
PINNS_based_SOL_trainer(
    field=phi_w,
    physical_model=model,
    statistical_model=st_model,
    optimizer=optimizer,
    simulation_steps=3,
    time_step=0.1,
    loss=lambda x,y: (x-y)**2
)